In [141]:
import re
from typing import Tuple

import polars as pl

In [142]:
ORDINAL_STREET_REGEX = r"E\. \d{2}[a-z]{2} St."


def list_to_parsed_set(unparsed_list: [str]):
    unparsed_list.sort()
    parsed_set = set()
    for element in unparsed_list:
        if "/" in element:
            for p in element.split(r" / "):
                fmt_element = p.strip().lower()
                if p:
                    parsed_set.add(fmt_element)
        else:
            fmt_element = element.strip().lower()
            parsed_set.add(fmt_element)
    return parsed_set


def create_street_tuple(street: str, blvd: bool = False) -> Tuple[str, str, str]:
    street_type = "Ave." if not blvd else "Blvd."

    return street, f"S. {street}", f"S. {street} {street_type}"


STREET_CORRECTIONS = [
    create_street_tuple("Blackstone"),
    create_street_tuple("Cottage Grove"),
    create_street_tuple("Cornell"),
    create_street_tuple("Dorchester"),
    create_street_tuple("Drexel"),
    create_street_tuple("East View Park"),
    create_street_tuple("East End"),
    create_street_tuple("Ellis"),
    create_street_tuple("Evans"),
    create_street_tuple("Everett"),
    create_street_tuple("Greenwood"),
    create_street_tuple("Harper"),
    create_street_tuple("Hyde Park", blvd=True),
    create_street_tuple("Ingleside"),
    create_street_tuple("Kenwood"),
    create_street_tuple("Kimbark"),
    create_street_tuple("Lake Park"),
    ("Lake Shore", "S. Lake Shore", "S. Lake Shore Dr."),
    ("Madison Park", "E. Madison Park", "E. Madison Park"),
    create_street_tuple("Maryland"),
    ("Morgan", "Morgan", "Morgan Dr."),
    create_street_tuple("Oakenwald"),
    create_street_tuple("Oakwood", blvd=True),
    ("Ridgewood", "S. Ridgewood", "S. Ridgewood Ct."),
    ("Rochdale", "E. Rochdale", "E. Rochdale Pl."),
    create_street_tuple("Stony Island"),
    create_street_tuple("University"),
    create_street_tuple("Woodlawn"),
]

In [143]:
# Import incidents and format columns
df = pl.read_csv(
    "./data/incident_dump.csv",
).with_columns(
    pl.col("reported")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%z")
    .dt.convert_time_zone("America/Chicago"),
    pl.col("reported_date").str.to_date("%Y-%m-%d"),
    pl.col("validated_location").str.split(",").cast(pl.List(pl.Float64)),
    pl.col("incident").str.to_lowercase(),
)

In [144]:
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2978
"""information""",1382
"""found property…",923
"""lost property""",839
"""traffic violat…",534
"""information / …",517
"""liquor law vio…",443
"""medical call""",431
"""battery""",425


In [145]:
print(list_to_parsed_set(df["incident"].to_list()))

{'resisting police', 'unlawful discharge of firearm', 'aggravated robbery', 'obstructing police', 'found person', 'chemical spill', 'sick person', 'threatening call', 'medical call', 'disorderly conduct', 'pedestrian', 'traffic and warrant arrest', 'suspicious person', 'attempted', 'possession of narcotic with intent to deliver', 'deceptive practice', 'alarm', 'death investigation', 'recovered property', 'unlawful possession of firearm', 'found weapon', 'unlawful use of a weapon', 'damage to city property', 'harassing telephone call', 'accidental fire', 'arson by fire', 'counterfeit check', 'suspect narcotic found', 'accidental', 'information criminal damage to property', 'non-criminal fire', 'forged prescription', 'suspicious mail', 'public indecency', 'hit and run', 'found', 'battery of police officer', 'aggravated battery of police officer', 'resisting arrest', 'other', 'trespass to land', 'unlawful discharge of a weapon', 'delayed report', 'battery to police officer', 'harassment v

In [146]:
excluded_list = [
    "Fondling",
    "Medical Call",
    "Luring a Minor",
    "Lost Property",
    "Stalking",
    "Sexual Assault",
    "Dating",
    "Stalking",
    "Domestic",
    "Sex",
    "Found Property",
    "Mental Health",
    "Harassment by Electronic Means",
    "Well-Being",
    "Threatening Phone Call",
    "Medical Transport",
    "Warrant",
    "Lost Wallet",
    "Fire Alarm",
    "Chemical Spill",
    "Suspicious Mail",
    "Eavesdropping",
    "Sex Offense",
    "Sex Offender",
    "Sex Crime",
    "Domestic Aggravated Battery",
    "Dating Violence",
    "Harassing Messages",
]
df = df.filter(~pl.col("incident").str.contains("|".join(excluded_list)))
df.groupby(["incident"]).agg(pl.count()).sort("count", descending=True)

incident,count
str,u32
"""theft""",2978
"""information""",1382
"""found property…",923
"""lost property""",839
"""traffic violat…",534
"""information / …",517
"""liquor law vio…",443
"""medical call""",431
"""battery""",425


In [147]:
print(list_to_parsed_set(df["incident"].to_list()))

{'resisting police', 'unlawful discharge of firearm', 'aggravated robbery', 'obstructing police', 'found person', 'chemical spill', 'sick person', 'threatening call', 'medical call', 'disorderly conduct', 'pedestrian', 'traffic and warrant arrest', 'suspicious person', 'attempted', 'possession of narcotic with intent to deliver', 'deceptive practice', 'alarm', 'death investigation', 'recovered property', 'unlawful possession of firearm', 'found weapon', 'unlawful use of a weapon', 'damage to city property', 'harassing telephone call', 'accidental fire', 'arson by fire', 'counterfeit check', 'suspect narcotic found', 'accidental', 'information criminal damage to property', 'non-criminal fire', 'forged prescription', 'suspicious mail', 'public indecency', 'hit and run', 'found', 'battery of police officer', 'aggravated battery of police officer', 'resisting arrest', 'other', 'trespass to land', 'unlawful discharge of a weapon', 'delayed report', 'battery to police officer', 'harassment v

In [148]:
len(list_to_parsed_set(df["incident"].to_list()))

392

In [149]:
df

comments,disposition,incident,location,occurred,predicted_incident,reported,reported_date,ucpd_id,validated_address,validated_location
str,str,str,str,str,str,"datetime[μs, America/Chicago]",date,str,str,list[f64]
"""Wallet taken f…","""Referred""","""information / …","""1340 E. 47th S…","""2/14/24 4:00 P…",null,2024-02-15 09:48:00 CST,2024-02-15,"""2024-006099""","""1340 E 47TH ST…","[41.809741, -87.592793]"
"""A subject took…","""Referred""","""information / …","""1554 E. 55th S…","""2/15/24 11:00 …",null,2024-02-15 11:02:00 CST,2024-02-15,"""2024-006107""","""1554 E 55TH ST…","[41.795293, -87.588745]"
"""Window smashed…","""Referred""","""information / …","""5510 S. Kimbar…","""2/15/24 2:00 P…",null,2024-02-15 17:01:00 CST,2024-02-15,"""2024-006148""","""5510 S KIMBARK…","[41.794563, -87.594988]"
"""A suspicious v…","""Closed""","""information""","""950 E. 59th St…","""2/14/24 11:30 …",null,2024-02-15 10:12:00 CST,2024-02-15,"""24-00151""","""950 E 59TH ST,…","[41.78779, -87.602334]"
"""Unattended, un…","""Open""","""theft""","""860 E. 59th St…","""2/15/24 12:00 …",null,2024-02-15 12:07:00 CST,2024-02-15,"""24-00152""","""860 E 59TH ST,…","[41.787775, -87.60426]"
"""Three unknown …","""Referred""","""information / …","""5100 S. Dorche…","""2/14/24 7:45 P…",null,2024-02-14 20:18:00 CST,2024-02-14,"""2024-006015""","""5100 S DORCHES…","[41.80236, -87.591927]"
"""Person walking…","""CPD""","""information / …","""5428 S. Kimbar…","""2/14/24 9:05 P…",null,2024-02-14 21:09:00 CST,2024-02-14,"""2024-006021""","""5428 S KIMBARK…","[41.79714, -87.595049]"
"""A 2008 Kia Sou…","""CPD""","""information / …","""5428 S. Kimbar…","""2/14/24 9:05 P…",null,2024-02-14 21:34:00 CST,2024-02-14,"""2024-006025""","""5428 S KIMBARK…","[41.79714, -87.595049]"
"""iPad and Air P…","""Open""","""theft""","""1131 E. 57th S…","""2/13/24 to 2/1…",null,2024-02-14 10:05:00 CST,2024-02-14,"""24-00148""","""1131 E 57TH ST…","[41.791368, -87.598328]"


In [150]:
df.filter(pl.col("location").str.contains(" between ")).filter(
    ~pl.col("location").str.contains(" to ")
)

comments,disposition,incident,location,occurred,predicted_incident,reported,reported_date,ucpd_id,validated_address,validated_location
str,str,str,str,str,str,"datetime[μs, America/Chicago]",date,str,str,list[f64]
"""Two unknown su…","""Referred""","""information / …","""S. Dorchester …","""2/9/24 12:10 P…",null,2024-02-09 12:18:00 CST,2024-02-09,"""2024-005301""","""between E. 51s…","[41.800894, -87.59668]"
"""Person reports…","""Referred""","""information / …","""S. Blackstone …","""1/25/24 2:40 P…",null,2024-02-07 12:04:00 CST,2024-02-07,"""2024-005040""","""and S. Blackst…","[41.7933, -87.592458]"
"""Package taken …","""Referred""","""information / …","""S. Ellis Ave. …","""2/6/24 5:00 PM…",null,2024-02-07 14:22:00 CST,2024-02-07,"""2024-005052""","""and S. Ellis A…","[41.795196, -87.586625]"
"""Victim reporte…","""CPD""","""information / …","""S. East End Av…","""10/20/23 3:00 …",null,2024-02-07 17:23:00 CST,2024-02-07,"""24-00130""","""between E. 50t…","[41.802257, -87.605867]"
"""Suspect observ…","""CPD""","""information / …","""E. 54th St. be…","""2/4/24 1:20 PM…",null,2024-02-04 13:22:00 CST,2024-02-04,"""2024-004615""","""between S. Woo…","[41.793722, -87.5949]"
"""A person walki…","""CPD""","""information / …","""S. Dorchester …","""2/2/24 10:45 P…",null,2024-02-02 23:13:00 CST,2024-02-02,"""2024-004492""","""between E. 51s…","[41.800894, -87.59668]"
"""Unknown person…","""CPD""","""information / …","""S. Kenwood Ave…","""1/30/24 7:30 A…",null,2024-01-30 20:33:00 CST,2024-01-30,"""2024-004004""","""and South Kenw…","[41.791219, -87.606736]"
"""Unknown person…","""CPD""","""information / …","""S. Kimbark Ave…","""1/30/24 8:15 P…",null,2024-01-30 20:46:00 CST,2024-01-30,"""2024-004007""","""between E. 53r…","[41.798003, -87.594163]"
"""Person reports…","""Closed""","""lost property""","""S. Ellis Ave. …","""1/30/24 6:00 P…",null,2024-01-30 22:07:00 CST,2024-01-30,"""24-00112""","""between E. 55t…","[41.794767, -87.591675]"


In [151]:
df.filter(pl.col("location").str.contains(" to "))

comments,disposition,incident,location,occurred,predicted_incident,reported,reported_date,ucpd_id,validated_address,validated_location
str,str,str,str,str,str,"datetime[μs, America/Chicago]",date,str,str,list[f64]
"""Person reporte…","""Closed""","""lost property""","""5100 S. Drexel…","""2/5/24 8:00 AM…",null,2024-02-13 13:50:00 CST,2024-02-13,"""24-00144""","""5100 South Dre…","[41.793421, -87.583905]"
"""Electronic har…","""Open""","""harassment by …","""E. 61st St. be…","""11/30/23 to 12…",null,2023-12-28 10:32:00 CST,2023-12-28,"""23-01269""","""between S. Gre…","[41.79506, -87.596557]"
"""Two suitcases …","""Referred""","""information / …","""S. Stony Islan…","""8/30/23 to 8/3…",null,2023-08-31 14:06:00 CDT,2023-08-31,"""2023-030412""","""to S. Stony Is…","[41.789607, -87.596414]"
"""Person reporte…","""Closed""","""lost property""","""1330 E. 53rd S…","""8/29/23 11:30 …",null,2023-08-29 15:06:00 CDT,2023-08-29,"""23-00810""","""1330 East 53rd…","[41.795231, -87.587464]"
"""Person lost wa…","""Closed""","""lost property""","""1100 E. 57th S…","""4/17/23 10:00 …",null,2023-04-24 14:39:00 CDT,2023-04-24,"""23-00386""","""1100 E. 57th S…","[41.796189, -87.588537]"
"""Person reports…","""Closed""","""lost property""","""1101 E. 56th S…","""2/20/23 to 2/2…",null,2023-02-21 16:34:00 CST,2023-02-21,"""23-00171""","""1101 E. 56th S…","[41.792279, -87.599954]"
"""Person lost wr…","""Closed""","""lost property""","""929 E. 57th St…","""12/13/21 8:45 …",null,2021-12-16 16:25:00 CST,2021-12-16,"""21-00908""","""to E. 57th and…","[41.791374, -87.601302]"
"""Person reports…","""Closed""","""lost wallet""","""5100 S. Blacks…","""12/11/21 4:02 …",null,2021-12-11 16:02:00 CST,2021-12-11,"""21-00889""","""5100 South Bla…","[41.789735, -87.601538]"
"""Person reporte…","""Closed""","""lost property""","""5400 S. Shore …","""7/6/21 1:30 AM…",null,2021-07-06 20:38:00 CDT,2021-07-06,"""21-00391""","""5400 S. Shore …","[41.791024, -87.604903]"


In [152]:
STREET_CORRECTIONS_FINAL = [s for _, _, s in STREET_CORRECTIONS]
STREET_CORRECTIONS_FINAL.extend(
    ["S. Shore Dr.", "Midway Plaisance", "E. Drexel Sq.", "S. Payne Dr.", "E. Park Pl."]
)


def parse_between_address(addr: str) -> None:
    ordinal_streets = re.findall(r"E\. \d{2}[a-z]{2} \w+", addr)
    ordinal_streets.sort()
    ordinals = list(map(int, re.findall(r"E\. (\d{2})[a-z]{2} \w+", addr)))
    ordinals.sort()
    non_ordinal_streets = [s for s in STREET_CORRECTIONS_FINAL if s in addr]

    if len(ordinals) == 2 and ordinals[1] - ordinals[0] != 1:
        print(addr)
        print(f"Large ordinals: {ordinals}")
    elif len(ordinals) == 2 and len(non_ordinal_streets) == 1:
        if ordinals[1] - ordinals[0] != 1:
            print(addr)
            print(ordinal_streets, non_ordinal_streets)
        return
        # print(f"{ordinals[0]}20 {non_ordinal_streets[0]}")
    elif len(non_ordinal_streets) > 1 and len(ordinals) == 1:
        return
        # print(f"{non_ordinal_streets[0]} and {ordinal_streets[0]}")
        # print(f"{non_ordinal_streets[1]} and {ordinal_streets[0]}")
    else:
        print(addr)
        print(ordinal_streets, non_ordinal_streets)
    print("----")

In [153]:
# between_addr = (
#     df.filter(pl.col("location").str.contains("between "))
#     .filter(~pl.col("location").str.contains(" to "))["location"]
#     .to_list()
# )

# Do NOT geocode if it has the word 'to'

between_addr = df.filter(pl.col("location").str.contains(" to "))["location"].to_list()

In [154]:
print(len(between_addr))
for i in range(len(between_addr)):
    parse_between_address(between_addr[i])

84
5100 S. Drexel Ave. to 5700 S. Hyde Park Blvd.
[] ['S. Drexel Ave.', 'S. Hyde Park Blvd.']
----
1330 E. 53rd St. to E. 55th St. and S. Lake Park Ave. (Public Way)
Large ordinals: [53, 55]
----
1100 E. 57th St. to 1526 E. 55th St.
Large ordinals: [55, 57]
----
1101 E. 56th St. to 1100 E. 57th St.
['E. 56th St', 'E. 57th St'] []
----
929 E. 57th St. to E. 57th and S. Ellis Ave.
Large ordinals: [57, 57]
----
5100 S. Blackstone Ave. to 970 E. 58th St.
['E. 58th St'] ['S. Blackstone Ave.']
----
5400 S. Shore to 5700 S. Maryland Ave.
[] ['S. Maryland Ave.']
----
E. 57th St. and S. Maryland Ave. to E. 57th St.and S. Kenwood Ave.
Large ordinals: [57, 57]
----
E. 60th St. and S. Ellis Ave. to E. 55th St. and S. University Ave.
Large ordinals: [55, 60]
----
1100 E. 57th St. to 900 E. 60th St.
Large ordinals: [57, 60]
----
929 E. 57th St. to 1200 E. 53rd St.
Large ordinals: [53, 57]
----
between E. 61st St. and S. Cottage Grove Ave. to E. 58th St. and S. Greenwood Ave.
Large ordinals: [58, 61]

In [155]:
df.filter(~pl.col("location").str.contains(" to ")).filter(
    pl.col("location").str.to_lowercase().str.contains("between ")
).head()

comments,disposition,incident,location,occurred,predicted_incident,reported,reported_date,ucpd_id,validated_address,validated_location
str,str,str,str,str,str,"datetime[μs, America/Chicago]",date,str,str,list[f64]
"""Two unknown su…","""Referred""","""information / …","""S. Dorchester …","""2/9/24 12:10 P…",null,2024-02-09 12:18:00 CST,2024-02-09,"""2024-005301""","""between E. 51s…","[41.800894, -87.59668]"
"""Person reports…","""Referred""","""information / …","""S. Blackstone …","""1/25/24 2:40 P…",null,2024-02-07 12:04:00 CST,2024-02-07,"""2024-005040""","""and S. Blackst…","[41.7933, -87.592458]"
"""Package taken …","""Referred""","""information / …","""S. Ellis Ave. …","""2/6/24 5:00 PM…",null,2024-02-07 14:22:00 CST,2024-02-07,"""2024-005052""","""and S. Ellis A…","[41.795196, -87.586625]"
"""Victim reporte…","""CPD""","""information / …","""S. East End Av…","""10/20/23 3:00 …",null,2024-02-07 17:23:00 CST,2024-02-07,"""24-00130""","""between E. 50t…","[41.802257, -87.605867]"
"""Suspect observ…","""CPD""","""information / …","""E. 54th St. be…","""2/4/24 1:20 PM…",null,2024-02-04 13:22:00 CST,2024-02-04,"""2024-004615""","""between S. Woo…","[41.793722, -87.5949]"


In [156]:
df.filter(
    pl.col("location").str.contains("|".join([" and ", "between ", " to ", " at "]))
)

comments,disposition,incident,location,occurred,predicted_incident,reported,reported_date,ucpd_id,validated_address,validated_location
str,str,str,str,str,str,"datetime[μs, America/Chicago]",date,str,str,list[f64]
"""Person reporte…","""Closed""","""lost property""","""5100 S. Drexel…","""2/5/24 8:00 AM…",null,2024-02-13 13:50:00 CST,2024-02-13,"""24-00144""","""5100 South Dre…","[41.793421, -87.583905]"
"""Two unknown su…","""Referred""","""information / …","""S. Dorchester …","""2/9/24 12:10 P…",null,2024-02-09 12:18:00 CST,2024-02-09,"""2024-005301""","""between E. 51s…","[41.800894, -87.59668]"
"""Person reports…","""Referred""","""information / …","""S. Blackstone …","""1/25/24 2:40 P…",null,2024-02-07 12:04:00 CST,2024-02-07,"""2024-005040""","""and S. Blackst…","[41.7933, -87.592458]"
"""Package taken …","""Referred""","""information / …","""S. Ellis Ave. …","""2/6/24 5:00 PM…",null,2024-02-07 14:22:00 CST,2024-02-07,"""2024-005052""","""and S. Ellis A…","[41.795196, -87.586625]"
"""Victim reporte…","""CPD""","""information / …","""S. East End Av…","""10/20/23 3:00 …",null,2024-02-07 17:23:00 CST,2024-02-07,"""24-00130""","""between E. 50t…","[41.802257, -87.605867]"
"""Suspect observ…","""CPD""","""information / …","""E. 54th St. be…","""2/4/24 1:20 PM…",null,2024-02-04 13:22:00 CST,2024-02-04,"""2024-004615""","""between S. Woo…","[41.793722, -87.5949]"
"""A person walki…","""CPD""","""information / …","""S. Dorchester …","""2/2/24 10:45 P…",null,2024-02-02 23:13:00 CST,2024-02-02,"""2024-004492""","""between E. 51s…","[41.800894, -87.59668]"
"""Unknown person…","""CPD""","""information / …","""S. Kenwood Ave…","""1/30/24 7:30 A…",null,2024-01-30 20:33:00 CST,2024-01-30,"""2024-004004""","""and South Kenw…","[41.791219, -87.606736]"
"""Unknown person…","""CPD""","""information / …","""S. Kimbark Ave…","""1/30/24 8:15 P…",null,2024-01-30 20:46:00 CST,2024-01-30,"""2024-004007""","""between E. 53r…","[41.798003, -87.594163]"


In [157]:
df.shape

(16883, 11)

In [159]:
to_cnt = 0
single_and_cnt = 0
between_cnt = 0
between_multi_and_cnt = 0
mult_and_cnt = 0
non_cat = 0

bad_locations = df.filter(
    pl.col("location")
    .str.to_lowercase()
    .str.contains("|".join([" and ", "between ", " to ", " at "]))
)["location"].to_list()

for bl in bad_locations:
    bl = bl.split(" (")[0] if "(" in bl else bl
    bl = bl.replace(")", "")
    and_cnt = len([s for s in bl.split() if s == "and"])

    if " to " in bl or " or " in bl:
        # Toss these out
        to_cnt += 1
    elif "between " in bl:
        between_cnt += 1
        ordinal_streets = re.findall(r"E\. \d{2}[a-z]{2} \w+", bl)
        ordinals = list(map(int, re.findall(r"E\. (\d{2})[a-z]{2} \w+", bl)))
        ordinals.sort()
        non_ordinal_streets = [s for s in STREET_CORRECTIONS_FINAL if s in bl]
        if and_cnt > 1:
            # Toss these out
            between_multi_and_cnt += 1
        else:
            if len(ordinals) == 2 and len(non_ordinal_streets) == 1:
                # print(f"{ordinals[0]}20 {non_ordinal_streets[0]}")
                # print("NOW WE AVERAGE THINGS OUT")
                1 + 1
            elif len(ordinals) == 1 and len(non_ordinal_streets) == 2:
                # print(f"{ordinals[0]}00 {non_ordinal_streets[0]}")
                # print(f"{ordinals[0]}00 {non_ordinal_streets[1]}")
                1 + 1
            elif len(non_ordinal_streets) == 2 and and_cnt == 1:
                between_split = bl.split(" between ")
                verticals = between_split[1].split(" and ")
                print(f"{between_split[0]} and {verticals[0]}")
                print(f"{between_split[0]} and {verticals[1]}")

    elif and_cnt == 1 or " at " in bl:
        single_and_cnt += 1
    elif and_cnt > 1:
        # Toss these out
        mult_and_cnt += 1
    else:
        # Toss these out
        non_cat += 1

print(f"Total count: {len(bad_locations)}")
print(f"To count: {to_cnt}")
print(f"Between count: {between_cnt}")
print(f"Between and multi-and count: {between_multi_and_cnt}")
print(f"Single 'and' count: {single_and_cnt}")
print(f"Multiple 'and' count: {mult_and_cnt}")
print(f"Non-categorized count: {non_cat}")

E. Hyde Park Blvd. and S. Drexel Ave.
E. Hyde Park Blvd. and S. Ellis Ave.
E. Hyde Park Blvd. and S. Cornell Ave.
E. Hyde Park Blvd. and S. East End Ave.
E. Hyde Park Blvd. and S. Greenwood Ave.
E. Hyde Park Blvd. and S. University Ave.
E. Hyde Park Blvd. and S. Greenwood Ave.
E. Hyde Park Blvd. and S. University Ave.
E. Hyde Park Blvd. and S. Kenwood Ave.
E. Hyde Park Blvd. and S. Dorchester Ave.
E. Hyde Park Blvd. and S. Ellis Ave.
E. Hyde Park Blvd. and S. Greenwood Ave.
E. Hyde Park Blvd. and S. Woodlawn Ave.
E. Hyde Park Blvd. and S. Kimbark Ave.
E. Hyde Park Blvd. and S. Dorchester Ave.
E. Hyde Park Blvd. and S. Blackstone Ave.
E. Hyde Park Blvd. and S. Dorchester Ave.
E. Hyde Park Blvd. and S. Blackstone Ave.
E. Hyde Park Blvd. and S. Woodlawn Ave.
E. Hyde Park Blvd. and S. Kimbark Ave.
E. Hyde Park Blvd. and S. Kimbark Ave.
E. Hyde Park Blvd. and S. Kenwood Ave.
E. Hyde Park Blvd. and S. Greenwood Ave.
E. Hyde Park Blvd. and S. University Ave.
E. Hyde Park Blvd. and S. Kenwood 